<a href="https://colab.research.google.com/github/VivekanandaMudelli/hand_drawn_sketch_recognition/blob/main/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import gdown
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
import joblib  # For saving the model
from tqdm import tqdm

In [2]:
# file_id = "1EFQlS9C4YGT5CNjD_NX_Q-HaiyLH4sjE"
file_id = "14X_8QdPW12hGHnNzZztuz3keHOo9Cq4q"

# Construct the direct download URL
# https://drive.google.com/file/d/14X_8QdPW12hGHnNzZztuz3keHOo9Cq4q/view?usp=sharing
download_url = f"https://drive.google.com/uc?id={file_id}"

# Download the file
output = "data.csv"  # Change filename as needed
gdown.download(download_url, output, quiet=False)

# Read the CSV file
data = pd.read_csv(output)
data = data.dropna()

# X = data.drop(columns = ['label'])
# y = data['label']
X = data.drop(data.columns[0],axis = 1).drop(columns = ['encoded_part','extracted_part'])
y = data['encoded_part']

print(X.shape)
print(y.shape)

Downloading...
From (original): https://drive.google.com/uc?id=14X_8QdPW12hGHnNzZztuz3keHOo9Cq4q
From (redirected): https://drive.google.com/uc?id=14X_8QdPW12hGHnNzZztuz3keHOo9Cq4q&confirm=t&uuid=08608c42-bf6f-4e0c-ab6b-25b8f9eabb89
To: /content/data.csv
100%|██████████| 793M/793M [00:11<00:00, 67.4MB/s]


(20000, 2048)
(20000,)


In [3]:
def sigmoid(Z):
  return 1 / (1 + np.exp(-np.array(Z)))

def sigmoid_derivative(Z):
  return sigmoid(Z) * (1 - sigmoid(Z))

def ReLU(Z):
  return np.maximum(0, np.array(Z))

def ReLU_derivative(Z):
    return Z > 0

def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=1, keepdims=True))  # stability fix
    return expZ / np.sum(expZ, axis=1, keepdims=True)

def linear(Z):
  return Z

def linear_derivative(Z):
  return np.ones(Z.shape)

def mse(y_true, y_pred):
  return np.mean(np.power(y_true - y_pred, 2))

def cross_entropy_loss(y_true, y_pred):
    m = y_true.shape[0]
    # Avoid log(0) by adding small value
    log_likelihood = -np.log(y_pred[range(m), y_true.argmax(axis=1)] + 1e-9)
    return np.sum(log_likelihood) / m

def accuracy(y_true, y_pred):
    return np.mean(np.argmax(y_true, axis=1) == np.argmax(y_pred, axis=1))

In [4]:
import numpy as np
from tqdm import tqdm

class NeuralNetwork:
    def __init__(self, layers, activation='relu', learning_rate=0.01):
        self.layers = layers
        self.activation = activation
        self.learning_rate = learning_rate
        self.weights = []
        self.biases = []

        for i in range(len(layers) - 1):
            # Xavier Initialization
            self.weights.append(np.random.randn(layers[i+1], layers[i]) * np.sqrt(1. / layers[i]))
            self.biases.append(np.zeros((layers[i+1])))

    def forward(self, x):
        activations = [np.array(x)]
        z_values = []

        for i in range(len(self.weights)):
            z = np.dot(activations[-1], self.weights[i].T) + self.biases[i]
            z_values.append(z)

            if i == len(self.weights) - 1:
                activations.append(softmax(z))  # Final layer softmax
            elif self.activation == 'sigmoid':
                activations.append(sigmoid(z))
            elif self.activation == 'relu':
                activations.append(ReLU(z))
            elif self.activation == 'linear':
                activations.append(linear(z))

        return activations, z_values

    def backward(self, x, y, activations, z_values):
        m = y.shape[0] if y.ndim > 1 else 1
        delta = activations[-1] - y  # Cross-entropy derivative

        for i in range(len(self.weights) - 1, -1, -1):
            if i < len(self.weights) - 1:
                if self.activation == 'sigmoid':
                    delta *= sigmoid_derivative(z_values[i])
                elif self.activation == 'relu':
                    delta *= ReLU_derivative(z_values[i])
                elif self.activation == 'linear':
                    delta *= linear_derivative(z_values[i])

            a_prev = activations[i] if activations[i].ndim > 1 else activations[i].reshape(1, -1)
            delta_mat = delta if delta.ndim > 1 else delta.reshape(1, -1)

            self.weights[i] -= self.learning_rate * (delta_mat.T @ a_prev) / m
            self.biases[i] -= self.learning_rate * delta_mat.sum(axis=0) / m

            delta = delta @ self.weights[i]

    def train(self, x_train, y_train, x_val=None, y_val=None, epochs=20, batch_size=64):
        for epoch in range(epochs):
            indices = np.arange(len(x_train))
            np.random.shuffle(indices)
            x_train, y_train = x_train[indices], y_train[indices]

            for start in range(0, len(x_train), batch_size):
                end = start + batch_size
                x_batch = x_train[start:end]
                y_batch = y_train[start:end]

                activations, z_values = self.forward(x_batch)
                self.backward(x_batch, y_batch, activations, z_values)

            # Evaluate
            train_pred = self.predict(x_train)
            train_loss = cross_entropy_loss(y_train, train_pred)
            train_acc = accuracy(y_train, train_pred)

            if x_val is not None and y_val is not None:
                val_pred = self.predict(x_val)
                val_loss = cross_entropy_loss(y_val, val_pred)
                val_acc = accuracy(y_val, val_pred)
                print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
            else:
                print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")

    def predict(self, x_test):
        activations, _ = self.forward(x_test)
        return activations[-1]


In [5]:
# Encode labels
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_onehot = np.eye(250)[y_encoded]  # One-hot encoding

# Train/val/test split
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42, stratify=y_encoded)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
Layers = np.array([X_train.shape[1],1024,512,250])
ANN = NeuralNetwork(Layers,activation = 'relu',learning_rate=0.005)
ANN.train(X_train,y_train, x_val=None, y_val=None, epochs=60, batch_size=64) # epochs=100, batch_size=64 (or any desired batch_size)

Epoch 1/60 | Train Loss: 5.0733 | Train Acc: 0.0852
Epoch 2/60 | Train Loss: 4.4740 | Train Acc: 0.1875
Epoch 3/60 | Train Loss: 3.8586 | Train Acc: 0.2768
Epoch 4/60 | Train Loss: 3.3489 | Train Acc: 0.3473
Epoch 5/60 | Train Loss: 2.9605 | Train Acc: 0.3985
Epoch 6/60 | Train Loss: 2.6666 | Train Acc: 0.4439
Epoch 7/60 | Train Loss: 2.4381 | Train Acc: 0.4749
Epoch 8/60 | Train Loss: 2.2637 | Train Acc: 0.5013
Epoch 9/60 | Train Loss: 2.1110 | Train Acc: 0.5257
Epoch 10/60 | Train Loss: 1.9893 | Train Acc: 0.5463
Epoch 11/60 | Train Loss: 1.8836 | Train Acc: 0.5637
Epoch 12/60 | Train Loss: 1.7826 | Train Acc: 0.5859
Epoch 13/60 | Train Loss: 1.7047 | Train Acc: 0.5979
Epoch 14/60 | Train Loss: 1.6196 | Train Acc: 0.6190
Epoch 15/60 | Train Loss: 1.5515 | Train Acc: 0.6354
Epoch 16/60 | Train Loss: 1.4977 | Train Acc: 0.6418
Epoch 17/60 | Train Loss: 1.4337 | Train Acc: 0.6567
Epoch 18/60 | Train Loss: 1.3791 | Train Acc: 0.6694
Epoch 19/60 | Train Loss: 1.3259 | Train Acc: 0.6813
Ep

In [9]:
y_pred = ANN.predict(X_test)

In [10]:
# Calculate accuracy
y_pred_labels = np.argmax(y_pred, axis=1)  # Convert probabilities to class labels
y_true_labels = np.argmax(y_test, axis=1)  # Convert one-hot to class labels
accuracy = accuracy_score(y_true_labels, y_pred_labels)
print(f"Test Accuracy: {accuracy:.4f}")

# Detailed performance metrics
# print("\nClassification Report:")
# print(classification_report(y_test, y_pred, target_names=label_encoder.classes_.astype(str)))

Test Accuracy: 0.5613
